<a href="https://colab.research.google.com/github/gustavolenin/Curso---Machine-Learning---Udemy/blob/main/Fraud_Detection_in_Electricity_and_Gas_Consumption_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Definição do Problema

Este projeto consiste em uma competição disponível no site da Zindi.

**Descrição do Desafio**:

Este desafio foi pensado especificamente para o AI Tunisia Hack 2019, que acontece de 20 a 22 de setembro. Bem-vindo aos participantes do AI Tunisia Hack!

Após o AI Hack Tunísia, esta competição será reaberta como um Knowledge Challenge para permitir que outros membros da comunidade Zindi aprendam e testem suas habilidades.

A Companhia Tunisina de Eletricidade e Gás (STEG) é uma empresa pública e não administrativa, responsável pelo fornecimento de eletricidade e gás em toda a Tunísia. A empresa sofreu enormes perdas na ordem de 200 milhões de dinares tunisianos devido a manipulações fraudulentas de medidores por parte dos consumidores.

Usando o histórico de cobrança do cliente, o objetivo do desafio é detectar e reconhecer clientes envolvidos em atividades fraudulentas.

A solução aumentará as receitas da empresa e reduzirá as perdas causadas por tais atividades fraudulentas.

-------------------------------------------------------------------------

**Sobre STEG** (https://www.steg.com.tn/en/institutionnel/mission.html):


A Companhia Tunisina de Eletricidade e Gás (STEG) é uma empresa pública e não administrativa. É responsável pelo fornecimento de eletricidade e gás em toda a Tunísia.

**Sobre o InstaDeep** (https://www.instadeep.com/):


A InstaDeep oferece sistemas de tomada de decisão baseados em IA para empresas. Com experiência em pesquisa de inteligência de máquina e implantações de negócios concretas, fornecemos uma vantagem competitiva aos nossos clientes em um mundo que prioriza a IA. Como uma das principais empresas de IA da África, a InstaDeep sabe em primeira mão do que o talento africano é realmente capaz.

A InstaDeep tem mais de 70 funcionários espalhados por sua sede em Londres e escritórios em Paris, Túnis, Nairóbi e Lagos. Além de suas conexões com instituições educacionais africanas, a empresa também possui fortes laços com escolas francesas de elite e universidades de primeira linha no Reino Unido.

**Sobre AI Hack Tunísia** (www.ai-hack-tunisia.com):


O AI Hack Tunisia 2019 é o maior Hackathon ML/AI já realizado na Tunísia, África e na região do Oriente Médio e Norte da África (MENA). Acontece de 20 a 22 de setembro. O evento é um hackathon duplo: a primeira parte é um Machine Learning Challenge individual e a segunda parte é uma competição em grupo focada em uma tecnologia específica.

# Extração dos Dados

In [ ]:
# Importação das Bibliotecas utilizadas
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score

# import warnings
# warnings.filterwarnings("ignore")

In [ ]:
df_client_train  = pd.read_csv('/content/client_train.csv')
df_invoice_train = pd.read_csv('/content/invoice_train.csv')
df_client_test   = pd.read_csv('/content/client_test.csv')
df_invoice_test  = pd.read_csv('/content/invoice_test.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# Identificando as colunas da cada um dos dataframes
print('---------------------------\ndf_client_train columns')
print(df_client_train.columns)
print('---------------------------\ndf_invoice_train columns')
print(df_invoice_train.columns)

---------------------------
df_client_train columns
Index(['disrict', 'client_id', 'client_catg', 'region', 'creation_date',
       'target'],
      dtype='object')
---------------------------
df_invoice_train columns
Index(['client_id', 'invoice_date', 'tarif_type', 'counter_number',
       'counter_statue', 'counter_code', 'reading_remarque',
       'counter_coefficient', 'consommation_level_1', 'consommation_level_2',
       'consommation_level_3', 'consommation_level_4', 'old_index',
       'new_index', 'months_number', 'counter_type'],
      dtype='object')


In [ ]:
# Unificando os datasets de treino
df_train = df_client_train.merge(df_invoice_train,on='client_id')

# Unificando os datasets de teste
df_test = df_client_test.merge(df_invoice_test,on='client_id')

In [ ]:
df_train.head()

,disrict,client_id,client_catg,region,creation_date,target,invoice_date,tarif_type,counter_number,counter_statue,...,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,60,train_Client_0,11,101,31/12/1994,0.0,2014-03-24,11,1335667,0,...,8,1,82,0,0,0,14302,14384,4,ELEC
1,60,train_Client_0,11,101,31/12/1994,0.0,2013-03-29,11,1335667,0,...,6,1,1200,184,0,0,12294,13678,4,ELEC
2,60,train_Client_0,11,101,31/12/1994,0.0,2015-03-23,11,1335667,0,...,8,1,123,0,0,0,14624,14747,4,ELEC
3,60,train_Client_0,11,101,31/12/1994,0.0,2015-07-13,11,1335667,0,...,8,1,102,0,0,0,14747,14849,4,ELEC
4,60,train_Client_0,11,101,31/12/1994,0.0,2016-11-17,11,1335667,0,...,9,1,572,0,0,0,15066,15638,12,ELEC


Um breve comentário a respeito da variável target: 

*   target = 1 - fraude
*   target = 0 - não fraude

In [ ]:
df_train.isnull().sum()

disrict                 0
client_id               0
client_catg             0
region                  0
creation_date           0
target                  0
invoice_date            0
tarif_type              0
counter_number          0
counter_statue          0
counter_code            0
reading_remarque        0
counter_coefficient     0
consommation_level_1    0
consommation_level_2    0
consommation_level_3    0
consommation_level_4    0
old_index               0
new_index               0
months_number           0
counter_type            0
dtype: int64

In [ ]:
df_train.dtypes

disrict                   int64
client_id                object
client_catg               int64
region                    int64
creation_date            object
target                  float64
invoice_date             object
tarif_type                int64
counter_number            int64
counter_statue           object
counter_code              int64
reading_remarque          int64
counter_coefficient       int64
consommation_level_1      int64
consommation_level_2      int64
consommation_level_3      int64
consommation_level_4      int64
old_index                 int64
new_index                 int64
months_number             int64
counter_type             object
dtype: object

In [ ]:
# Verificar valores únicos em cada variável
X_train.counter_statue.unique()

In [ ]:
df_train.drop(columns=['client_id','old_index', 'new_index'],inplace=True)
X_train = df_train.drop(columns=['target'])
y_train = df_train.target

# Partition data
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [ ]:
# Define categorical columns
categorical = list(X_train.select_dtypes('object').columns)
print(f"Categorical columns are: {categorical}")

# Define numerical columns
numerical = list(X_train.select_dtypes('number').columns)
print(f"Numerical columns are: {numerical}")

Categorical columns are: ['creation_date', 'invoice_date', 'counter_statue', 'counter_type']
Numerical columns are: ['disrict', 'client_catg', 'region', 'tarif_type', 'counter_number', 'counter_code', 'reading_remarque', 'counter_coefficient', 'consommation_level_1', 'consommation_level_2', 'consommation_level_3', 'consommation_level_4', 'months_number']


In [ ]:
# Define categorical pipeline
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

# Define numerical pipeline
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

# Combine categorical and numerical pipelines
preprocessor = ColumnTransformer([
    ('cat', cat_pipe, categorical),
    ('num', num_pipe, numerical)
])

# Fit a pipeline with transformers and an estimator to the training data
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LogisticRegression())
])
pipe.fit(X_train, y_train)

# Predict training data
y_train_pred = pipe.predict(X_train)
print(f"Predictions on training data: {y_train_pred}")

# Predict test data
y_test_pred = pipe.predict(X_test)
print(f"Predictions on test data: {y_test_pred}")

TypeError: ignored

In [ ]:
X_train.columns

Index(['disrict', 'client_catg', 'region', 'creation_date', 'invoice_date',
       'tarif_type', 'counter_number', 'counter_statue', 'counter_code',
       'reading_remarque', 'counter_coefficient', 'consommation_level_1',
       'consommation_level_2', 'consommation_level_3', 'consommation_level_4',
       'months_number', 'counter_type'],
      dtype='object')